In [1]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
import pymysql as a


In [ ]:
window =tk.Tk()
window.title("Face Detector")
window.config(background="light blue")

l1 =tk.Label(window,text="Name",font=("Algerian",20),bg='blue')
l1.grid(column=0,row=0)
t1= tk.Entry(window,width=50,bd=5)
t1.grid(column=1,row=0)

l2 =tk.Label(window,text="Age",font=("Algerian",20),bg='blue')
l2.grid(column=0,row=2)
t2= tk.Entry(window,width=50,bd=5)
t2.grid(column=1,row=2)

l3 =tk.Label(window,text="Address",font=("Algerian",20),bg='blue')
l3.grid(column=0,row=4)
t3= tk.Entry(window,width=50,bd=5)
t3.grid(column=1,row=4)


def generate_dataset():
    if (t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showerror("Error","Please fill all the fields")
    else:
        mydb = a.connect(
        host="localhost",
        user="root",
        password="Lokesh@123#",
        database ="autherized_user"
        )
        mycusor =mydb.cursor()
        mycusor.execute("SELECT * from Known_person")
        myresult = mycusor.fetchall()
        id=1
        for x in myresult:
            id+=1
        sql="insert into Known_person(id,Name,Age,Address) values(%s,%s,%s,%s)"
        val=(id,t1.get(),t2.get(),t3.get())
        mycusor.execute(sql,val)
        mydb.commit()

        face_classifier=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces=face_classifier.detectMultiScale(gray,1.3,5)
            #scaling factor=1.3
            #min neighbour=5
            if len(faces)==0:
                return None
            for(x,y,w,h) in faces:
                cropped_face=img[y:y+h,x:x+w]
            return cropped_face
        # Start video capture
        cap = cv2.VideoCapture(0)
        img_id = 0

        while True:
            ret,frame=cap.read()
            if face_cropped(frame) is not None:
                img_id+=1
                face=cv2.resize(face_cropped(frame),(200,200))
                face =cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)

                file_name_path=f"data/user.{id}.{img_id}.jpg"
                cv2.imwrite(file_name_path,face)
                cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                #(50,50)is the origin point from where text is to written
                #font scale =1
                #thickness=2
                cv2.imshow("face Cropped",face)
                if cv2.waitKey(1)==13 or int(img_id==200):
                    break
        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo("result","generating dataset is completed !!!")
    


b1=tk.Button(window,text="Generate dataset",font=("Algerian",20),bg='orange',fg='red',command=generate_dataset)
b1.grid(column=0,row=5)        

def train_classifier():
    data_dir="C:/Users/hseko/OneDrive/Desktop/face detector/data"
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    faces=[]
    ids=[]

    for image in path:
        img = Image.open(image).convert('L')
        imageNp=np.array(img,'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)
    ids= np.array(ids)

    #train the classifier and save
    clf =cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
    messagebox.showinfo("result","training dataset is completed")



b2=tk.Button(window,text="Training",font=("Algerian",20),bg='green',fg='white',command= train_classifier)
b2.grid(column=1,row=5)

def detect_face():
    def draw_boundary(img,classifier,scalefactor,minNeighbors,color,text,clf):
        gray_image=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image,scalefactor,minNeighbors)

        coords =[]
        for(x,y,w,h) in features:
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            id,pred = clf.predict(gray_image[y:y+h,x:x+w])
            confidence = int (100*(1-pred/300))
            mydb=a.connect(
            host="localhost",
            user="root",
            passwd="Lokesh@123#",
            database="autherized_user"
            )
            mycursor=mydb.cursor()
            mycursor.execute("select name from Known_person where id="+str(id))
            s = mycursor.fetchone()
            s = ''+''.join(s)

            if confidence>80:
                    cv2.putText(img,s,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
            else:
                    cv2.putText(img,"unknown",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
            coords=[x,y,w,h]
        return coords
    def recognize(img,clf,faceCascade):
        coords =draw_boundary(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
        return img
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf =cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture =cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        img = recognize(img, clf,faceCascade)
        cv2.imshow("Face detection",img)

        if cv2.waitKey(1)==13:
            break
    video_capture.release()
    cv2.destroyAllWindows()    
   
    

b3=tk.Button(window,text="Detect face",font=("Algerian",20),bg='pink',fg='black',command= detect_face)
b3.grid(column=2,row=5)

window.geometry("800x600")
window.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\hseko\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\hseko\AppData\Local\Temp\ipykernel_15260\495057124.py", line 146, in detect_face
    img = recognize(img, clf,faceCascade)
  File "C:\Users\hseko\AppData\Local\Temp\ipykernel_15260\495057124.py", line 136, in recognize
    coords =draw_boundary(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
  File "C:\Users\hseko\AppData\Local\Temp\ipykernel_15260\495057124.py", line 118, in draw_boundary
    mydb=a.connect(
    host="localhost",
    ...<2 lines>...
    database="autherized_user"
    )
  File "c:\Users\hseko\AppData\Local\Programs\Python\Python313\Lib\site-packages\pymysql\connections.py", line 361, in __init__
    self.connect()
    ~~~~~~~~~~~~^^
  File "c:\Users\hseko\AppData\Local\Programs\Python\Python313\Lib\site-packages\pymysql\co

In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk

# Create main window
window = tk.Tk()
window.title("Face Recognition System")
window.geometry("900x600")
window.config(background="#ddeeff")  # Light blue background

# Header Label
header = tk.Label(
    window,
    text="Face Detection System",
    font=("Algerian", 30, "bold"),
    bg="#1e90ff",  # Dodger blue
    fg="white",
    pady=10
)
header.pack(fill=tk.X)

# Create frames for layout organization
input_frame = tk.Frame(window, bg="#ddeeff", pady=20)
input_frame.pack(pady=20)

button_frame = tk.Frame(window, bg="#ddeeff")
button_frame.pack(pady=20)

# Input Fields
def create_label_entry(parent, label_text, row):
    label = tk.Label(parent, text=label_text, font=("Arial", 16, "bold"), bg="#ddeeff", fg="black")
    label.grid(row=row, column=0, padx=10, pady=10, sticky="w")
    entry = tk.Entry(parent, width=40, font=("Arial", 14), bd=3)
    entry.grid(row=row, column=1, padx=10, pady=10)
    return entry

t1 = create_label_entry(input_frame, "Name:", 0)
t2 = create_label_entry(input_frame, "Age:", 1)
t3 = create_label_entry(input_frame, "Address:", 2)

# Button Styles
btn_style = {
    "font": ("Arial", 16, "bold"),
    "bd": 3,
    "relief": tk.RAISED,
    "width": 15,
    "height": 2,
}
def generate_dataset():
    if (t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showerror("Error","Please fill all the fields")
    else:
        mydb = a.connect(
        host="localhost",
        user="root",
        password="Lokesh@123#",
        database ="project"
        )
        mycusor =mydb.cursor()
        mycusor.execute("SELECT * from Known_person")
        myresult = mycusor.fetchall()
        id=1
        for x in myresult:
            id+=1
        sql="insert into Known_person(id,Name,Age,Address) values(%s,%s,%s,%s)"
        val=(id,t1.get(),t2.get(),t3.get())
        mycusor.execute(sql,val)
        mydb.commit()

        face_classifier=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces=face_classifier.detectMultiScale(gray,1.3,5)
            #scaling factor=1.3
            #min neighbour=5
            if len(faces)==0:
                return None
            for(x,y,w,h) in faces:
                cropped_face=img[y:y+h,x:x+w]
            return cropped_face
        # Start video capture
        cap = cv2.VideoCapture(0)
        img_id = 0

        while True:
            ret,frame=cap.read()
            if face_cropped(frame) is not None:
                img_id+=1
                face=cv2.resize(face_cropped(frame),(200,200))
                face =cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)

                file_name_path=f"data/user.{id}.{img_id}.jpg"
                cv2.imwrite(file_name_path,face)
                cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                #(50,50)is the origin point from where text is to written
                #font scale =1
                #thickness=2
                cv2.imshow("face Cropped",face)
                if cv2.waitKey(1)==13 or int(img_id==200):
                    break
        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo("result","generating dataset is completed !!!")

# Buttons
b1 = tk.Button(
    button_frame,
    text="Generate Dataset",
    bg="#ffa500",  # Orange
    fg="white",
    command= generate_dataset,  # Add generate_dataset function
    **btn_style
)
b1.grid(row=0, column=0, padx=15, pady=10)
def train_classifier():
    data_dir="C:/Users/hseko/OneDrive/Desktop/face detector/data"
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    faces=[]
    ids=[]

    for image in path:
        img = Image.open(image).convert('L')
        imageNp=np.array(img,'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)
    ids= np.array(ids)

    #train the classifier and save
    clf =cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
    messagebox.showinfo("result","training dataset is completed")

b2 = tk.Button(
    button_frame,
    text="Train Classifier",
    bg="#32cd32",  # Lime Green
    fg="white",
    command=train_classifier,  # Add train_classifier function
    **btn_style
)
b2.grid(row=0, column=1, padx=15, pady=10)
def detect_face():
    def draw_boundary(img,classifier,scalefactor,minNeighbors,color,text,clf):
        gray_image=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image,scalefactor,minNeighbors)

        coords =[]
        for(x,y,w,h) in features:
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            id,pred = clf.predict(gray_image[y:y+h,x:x+w])
            confidence = int (100*(1-pred/300))
            mydb=a.connect(
            host="localhost",
            user="root",
            passwd="Lokesh@123#",
            database="autherized_user"
            )
            mycursor=mydb.cursor()
            mycursor.execute("select name from Known_person where id="+str(id))
            s = mycursor.fetchone()
            s = ''.join(s)

            if confidence>80:
                    cv2.putText(img,s,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
            else:
                    cv2.putText(img,"unknown",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0, 0, 255),1,cv2.LINE_AA)
            coords=[x,y,w,h]
        return coords
    def recognize(img,clf,faceCascade):
        coords =draw_boundary(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
        return img
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf =cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture =cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        img = recognize(img, clf,faceCascade)
        cv2.imshow("Face detection",img)

        if cv2.waitKey(1)==13:
            break
    video_capture.release()
    cv2.destroyAllWindows()    
b3 = tk.Button(
    button_frame,
    text="Detect Face",
    bg="#ff1493",  # Deep Pink
    fg="white",
    command=detect_face,  # Add detect_face function
    **btn_style
)
b3.grid(row=0, column=2, padx=15, pady=10)

# Footer
footer = tk.Label(
    window,
    text="Developed by [Lokesh kumar] - Face Detection System ©2024",
    font=("Arial", 12),
    bg="#1e90ff",
    fg="white",
    pady=10
)
footer.pack(side=tk.BOTTOM, fill=tk.X)


b4 = tk.Button(
    button_frame,
    text="Emotion detection",
    bg="#ffa500",  # Orange
    fg="white",
    command= "none",  # Add generate_dataset function
    **btn_style
)
b4.grid(row=1, column=0, padx=15, pady=10)

window.mainloop()